In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [2]:
from langchain.document_loaders import TextLoader

file_path = r"Text.txt"
loader = TextLoader(file_path)
documents = loader.load()

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=80, chunk_overlap=20)
doc_splitted = text_splitter.split_documents(documents)
doc_splitted

[Document(metadata={'source': 'Text.txt'}, page_content='HygeineWunder'),
 Document(metadata={'source': 'Text.txt'}, page_content='Staubsaugen war gestern!'),
 Document(metadata={'source': 'Text.txt'}, page_content='Das innovativste Luft- und Raumreinigungs-System für mehr Lebensqualität & ein'),
 Document(metadata={'source': 'Text.txt'}, page_content='& ein völlig neues Hygiene-Level in deinem Zuhause'),
 Document(metadata={'source': 'Text.txt'}, page_content='reinigt mit Wasser  perfekt für Allergiker  mehr als 15 Funktionen'),
 Document(metadata={'source': 'Text.txt'}, page_content='Eines der weltweit gesündesten & smartesten Reinigungs-Systeme – jetzt für'),
 Document(metadata={'source': 'Text.txt'}, page_content='– jetzt für deine eigenen 4 Wände sichern'),
 Document(metadata={'source': 'Text.txt'}, page_content='Reinigt & desinfiziert gleichzeitig. Feinstaub, Gase, Allergene, Bakterien,'),
 Document(metadata={'source': 'Text.txt'}, page_content='Bakterien, Keime, Viren und Milben

In [10]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(doc_splitted, embeddings)
retriever = vectorstore.as_retriever()

vectordb = Chroma.from_documents(doc_splitted, embeddings, persist_directory="./chroma_db")

c:\Users\parth\miniconda3\envs\genai\lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
system_message = """You are a helpful and professional chatbot for Hygienewunder, 
a German-based company specializing in hygiene and cleaning products. 
Your primary role is to assist customers with their inquiries and facilitate 
the sales process by:

Product Knowledge: Providing accurate and detailed information about Hygienewunder's products. 
Always extract relevant information from the official product details available at 
[https://www.hygienewunder.at/info/]. Ensure that you have comprehensive knowledge of 
each product's features, ingredients, and uses.

Product Recommendations: Based on user input (such as pet ownership, allergies, or specific 
cleaning needs), recommend one of the four available product packages. Engage users by asking 
clarifying questions to tailor your recommendations effectively.

For example, if a user mentions having pets, suggest the appropriate product package designed 
for pet owners.

Automated Sales Process: After making a recommendation, guide users through the purchasing 
process by providing a direct payment link (such as Stripe) to complete their order. Ensure 
that users understand the purchasing steps clearly.

No Discounts: Under no circumstances should you offer or mention any discounts, promotions, 
or special offers. The company does not authorize any discounts.

When communicating with customers:
- Be clear, polite, and concise in your responses.
- Use formal language in German (Sie) when responding to inquiries in German.
- Tailor your responses based on customer input, always ensuring that the information you 
  provide is helpful and user-friendly.
- Avoid making assumptions or providing information that deviates from the official product details."""

from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate

system_template = system_message + "\n\nContext: {context}"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "User: {question}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([
    system_message_prompt,
    human_message_prompt
])

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain_openai import OpenAI
from langchain.schema.runnable import RunnablePassthrough

llm = OpenAI(temperature=0.4)

# Create the question generator chain
question_generator = chat_prompt | llm

# Create the document chain
doc_chain = load_qa_chain(
    llm, 
    chain_type="stuff", 
    prompt=chat_prompt
)

# Create the conversational retrieval chain
conversation_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": chat_prompt}
)

C:\Users\parth\AppData\Local\Temp\ipykernel_4008\744677830.py:14: LangChainDeprecationWarning: This class is deprecated. See the following migration guides for replacements based on `chain_type`:
stuff: https://python.langchain.com/v0.2/docs/versions/migrating_chains/stuff_docs_chain
map_reduce: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_reduce_chain
refine: https://python.langchain.com/v0.2/docs/versions/migrating_chains/refine_chain
map_rerank: https://python.langchain.com/v0.2/docs/versions/migrating_chains/map_rerank_docs_chain

See also guides on retrieval and question-answering here: https://python.langchain.com/v0.2/docs/how_to/#qa-with-rag
  doc_chain = load_qa_chain(


In [7]:
def chat(query, chat_history=[]):
    result = conversation_chain({"question": query, "chat_history": chat_history})
    answer = result['answer'].strip()
    answer = answer.replace("System:", "").replace("Bot:", "").strip()
    chat_history.append((query, answer))
    return answer, chat_history

In [8]:
chat_history = []
while True:
    user_input = input("You: ")
    if user_input.lower() in ['exit', 'quit', 'bye']:
        break
    response, chat_history = chat(user_input, chat_history)
    print("Bot:", response)

C:\Users\parth\AppData\Local\Temp\ipykernel_4008\1072450301.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = conversation_chain({"question": query, "chat_history": chat_history})


Bot: and pet ownership.

 Based on your input, I would recommend our "Allergie Plus" package. It is specifically designed for allergy sufferers and pet owners, and is effective against dust mites and mold. It also helps save energy, making it a great choice for both your health and the environment. Would you like me to provide a direct payment link for this package?
Bot: Yes, there are four different packages available for purchase. Each package offers a unique combination of products to suit different needs and preferences. Would you like me to recommend a specific package for you based on your needs?
Bot: Of course! We have four packages available for purchase: Standard, Pet Owner, Allergy Sufferer, and Heavy Duty. Each package is designed to meet specific cleaning needs and comes with a variety of products. Would you like me to recommend a package based on your needs?
Bot: 
Bot: Yes, all of our products have been certified by the relevant authorities to ensure the highest quality an